In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

from spektral.datasets.citation import Citation
from spektral.layers.convolutional import GCNConv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
import os
import numpy as np
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [22]:
graph = Citation('cora')
X = graph.x
A = graph.a
Y = graph.y
# train_mask = graph.mask_tr
# val_mask = graph.mask_va
# test_mask = graph.mask_te

train_mask = graph.mask_te
val_mask = graph.mask_tr
test_mask = graph.mask_va

n_classes = Y.shape[-1]  #分类数
N = A.shape[0]  # 节点数
F = X.shape[-1]  # 每个节点的特征维数


In [23]:
print(np.shape(X), np.shape(Y), np.shape(A))
print(np.sum(train_mask), np.sum(val_mask), np.sum(test_mask))

(2708, 1433) (2708, 7) (2708, 2708)
1000 140 500


In [17]:
print(n_classes, Y[1])
print(N, F)

7 [0. 0. 0. 0. 1. 0. 0.]
2708 1433


In [18]:
X_in = Input(shape=(F, ))
A_in = Input(shape=(N, ), sparse=True)

X_1 = GCNConv(512, 'relu')([X_in, A_in])  # 一个节点的特征、一个节点和所有节点的邻接关系
X_1 = Dropout(0.5)(X_1)

# X_1 = GCNConv(512, 'relu')([X_1, A_in])  # 一个节点的特征、一个节点和所有节点的邻接关系
# X_1 = Dropout(0.5)(X_1)

X_1 = GCNConv(128, 'relu')([X_1, A_in])  # 一个节点的特征、一个节点和所有节点的邻接关系
X_1 = Dropout(0.5)(X_1)

X_1 = Dense(128, 'relu')(X_1)  # 一个节点的特征、一个节点和所有节点的邻接关系
X_1 = Dropout(0.5)(X_1)

X_2 = Dense(n_classes, 'softmax')(X_1) 
# X_2 = GCNConv(n_classes, 'softmax')([X_1, A_in])

model = Model(inputs=[X_in, A_in], outputs=X_2)
model.compile(optimizer='adam', loss='categorical_crossentropy', weighted_metrics=['acc'])
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 1433)]       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 2708)]       0                                            
__________________________________________________________________________________________________
gcn_conv_4 (GCNConv)            (None, 512)          734208      input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
dropout_6 (Dropout)             (None, 512)          0           gcn_conv_4[0][0]           

In [24]:
validation_data = ([X, A], Y, val_mask)
model.fit([X, A], Y, epochs=50, sample_weight=train_mask, validation_data=validation_data, batch_size=N, shuffle=False)

Epoch 1/50
1/1 [==============================] - 1s 516ms/step - loss: 0.5454 - acc: 0.5680 - val_loss: 0.0208 - val_acc: 0.9857
Epoch 2/50
1/1 [==============================] - 0s 97ms/step - loss: 0.5123 - acc: 0.6260 - val_loss: 0.0207 - val_acc: 0.9786
Epoch 3/50
1/1 [==============================] - 0s 113ms/step - loss: 0.5160 - acc: 0.6500 - val_loss: 0.0216 - val_acc: 0.9643
Epoch 4/50
1/1 [==============================] - 0s 105ms/step - loss: 0.4650 - acc: 0.6550 - val_loss: 0.0228 - val_acc: 0.9429
Epoch 5/50
1/1 [==============================] - 0s 110ms/step - loss: 0.4073 - acc: 0.6750 - val_loss: 0.0240 - val_acc: 0.9357
Epoch 6/50
1/1 [==============================] - 0s 204ms/step - loss: 0.4302 - acc: 0.6910 - val_loss: 0.0240 - val_acc: 0.9357
Epoch 7/50
1/1 [==============================] - 0s 174ms/step - loss: 0.4303 - acc: 0.6660 - val_loss: 0.0234 - val_acc: 0.9286
Epoch 8/50
1/1 [==============================] - 0s 189ms/step - loss: 0.4063 - acc: 0.688

In [25]:
eval_results = model.evaluate([X, A], Y, sample_weight=train_mask, batch_size=N)
print('Done. Train accuracy: {}'.format(eval_results))

1/1 [==============================] - 0s 227ms/step - loss: 0.0570 - acc: 0.9600
Done. Train accuracy: [0.05695149302482605, 0.9599999785423279]


In [26]:
eval_results = model.evaluate([X, A], Y, sample_weight=test_mask, batch_size=N)
print('Done. Test accuracy: {}'.format(eval_results))

1/1 [==============================] - 0s 66ms/step - loss: 0.1626 - acc: 0.8340
Done. Test accuracy: [0.1625974476337433, 0.8339999914169312]
